In [2]:
#Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import math
import scipy.optimize as optimize
import itertools
from scipy.stats import norm
from scipy.spatial.distance import cdist, pdist, squareform
from scipy.stats import qmc
import scipy.special
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel, ConstantKernel, DotProduct
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle
import gzip

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def calc_cs1_polynomial(true_model_coefficients, x):
    """
    Calculates the value of y for case study 1
    
    Parameters
    ----------
    true_model_coefficients: ndarray, The array containing the true values of Theta1 and Theta2
    x: ndarray, The list of xs that will be used to generate y
    
    Returns
    --------
    y_poly: ndarray, The noiseless values of y given theta_true and x
    """
    
    y_poly =  true_model_coefficients[0]*x + true_model_coefficients[1]*x**2 +x**3
       
    return y_poly

# ## define function that includes nonlinear model
# def model(theta,x):
#     '''
#     Toy model
#     Arguments:
#         theta: parameter vector
#         x: independent variable vector
#     Returns:
#         y_model: dependent variable prediction
#     '''
#     y_model = theta[0]*x + theta[1]*x**2 + x**3
#     return y_model

# ##New Cell

# # Create a function to optimize, in this case, least squares fitting
# def regression_func(theta, x, y):
#     '''
#     Function to define regression function for least-squares fitting
#     Arguments:
#         theta: parameter vector
#         x: independent variable vector
#         y: dependent variable vector (measurements)
#     Returns:
#         e: residual vector
#     '''
    
#     error = y - model(theta,x); #NOTE: Least squares will calculate sse based off this to minimize
    
#     return error
# #New Cell

# #Create a function to define the SSE for any Theta vector on a heat map.
# def sse_func(xx, yy, x, y):
#     '''
#     Function to define define sum of squared error function for heat map
#     Arguments:
#         xx: An N X D array of all Theta1 values
            
#         yy: An D X N array of all Theta2 values
#         theta: parameter vector
#         x: independent variable vector (predicted x values including noise)
#         y: dependent variable vector (predicted y values on Heat Map)
#     Returns:
#         sse: N x N sum of squared error matrix of all generated combination of xx and yy
#     '''
#     sse = np.zeros([len(xx),len(yy)])
    
#     for i in range(len(xx)):
#         for j in range(len(yy)):
#             theta = np.array([xx[i][j],yy[i][j]])
#             sse[i][j] = sum((y - model(theta,x))**2) 
    
#     return sse


def lhs_sampling(num_points, bounds, seed):
    """
    Design LHS Samples

    Parameters
    ----------
    num_points: int, number of points in LHS, should be greater than # of dimensions
    bounds: ndarray, array containing upper and lower bounds of elements in LHS sample. Defaults of 0 and 1
    seed: int, seed of random generation

    Returns
    -------
    lhs_data: ndarray, array of LHS sampling points with length (num_points) 
    """
    #Define number of dimensions
    dimensions = bounds.shape[1]
    #Define sampler
    sampler = qmc.LatinHypercube(d=dimensions, seed = seed)
    lhs_data = sampler.random(n=num_points)

    #Generate LHS data given bounds
    lhs_data = qmc.scale(lhs_data, bounds[0], bounds[1]) #Using this because I like that bounds can be different shapes

    return lhs_data



theta_names = ['theta_1', 'theta_2']
bounds_x_l = [-2]
bounds_x_u = [2]
bounds_theta_l = [-2, -2]
bounds_theta_u = [ 2,  2]
theta_ref = np.array([1.0, -1.0])   
noise_mean = 0
noise_std = 0.1

seed = 1

In [ ]:
np.random.seed(int(seed))

Xexp = np.linspace(-2,2,5)
Yexp = calc_cs1_polynomial(theta_ref, Xexp)

#Creates noise values with a certain stdev and mean from a normal distribution
noise = np.random.normal(size=len(Yexp), loc = noise_mean, scale = noise_std)
#Add noise to data
Yexp = Yexp + noise

# Evaluate model based on the assumed experimental values
X = np.linspace(np.min(Xexp),np.max(Xexp),100)
Y = calc_cs1_polynomial(theta_ref, X)

#Set GP Model
noise_kern = WhiteKernel(noise_level = noise_std**2, noise_level_bounds= "fixed") #bounds = "fixed"
c_kern = ConstantKernel(constant_value= np.ones(1), constant_value_bounds = (1e-3, 1e3))
Mat_kern = Matern(length_scale = 1, length_scale_bounds=(1e-3, 1e3), nu=2.5)
# RBF_kern = RBF(length_scale_bounds=(1e-14, 1e14))
# Dot_kern = DotProduct(sigma_0=1, sigma_0_bounds=(1e-14, 1e14))

kernel = c_kern*Mat_kern + noise_kern
gp_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=16, random_state = seed,
                                   optimizer = "fmin_l_bfgs_b")

fit_gp_model = gp_model.fit(Xexp.reshape(-1,1), Yexp)
Y_gp, stdev_gp = fit_gp_model.predict(X.reshape(-1,1), return_std=True)
__, cov_2 = fit_gp_model.predict(X.reshape(-1,1), return_cov = True)
print(fit_gp_model.kernel_)

In [ ]:
# Plot covariance
#Plot
XX_mesh = np.array(np.meshgrid(X,X))
X1, X2 = XX_mesh
z = cov_2
tot_lev = 150
#Set plot details for mean predictions
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (6,6))
ax = axes
title = "Covariance"
# title = ["Mean", "StDev"]
     
#Create a colormap and colorbar for each subplot
cs_fig = ax.contourf(X1, X2,z, levels = 900, cmap = "jet")
if np.amax(z) < 1e-1 or np.amax(z) > 1000:
    cbar = plt.colorbar(cs_fig, ax = ax, format='%.2e')
else:
    cbar = plt.colorbar(cs_fig, ax = ax, format = '%2.2f')

#Create a line contour for each colormap
cs2_fig = ax.contour(cs_fig, levels=cs_fig.levels[::tot_lev], colors='k', alpha=0.7, linestyles='dashed', linewidths=3)
ax.clabel(cs2_fig,  levels=cs_fig.levels[::tot_lev], fontsize=10, inline=1)
ax.scatter(Xexp,Xexp, color="green",s=25, label = "Training Data", marker = "o")  

#Get legend information
handles, labels = ax.get_legend_handles_labels()

#Plots axes such that they are scaled the same way (eg. circles look like circles) and name axes
ax.axis('scaled')  
ax.set_xlabel('$x_1$',weight='bold',fontsize=16)
ax.set_ylabel('$x_2$',weight='bold',fontsize=16)

#Plot title and set axis scale
ax.set_title(title, weight='bold',fontsize=16)
ax.set_xlim(left = np.amin(X1), right = np.amax(X1))
ax.set_ylim(bottom = np.amin(X2), top = np.amax(X2))      

#Plots legend and title
plt.tight_layout()
fig.legend(handles, labels, loc="upper left")  #bbox_to_anchor=(-0.01, 0.9), borderaxespad=0
plt.show()

In [ ]:
bounds = ([-np.inf, -np.inf], [np.inf, np.inf])
theta0 = np.random.rand(2) * (2+2) -2
Solution = optimize.least_squares(regression_func, theta0,bounds=bounds, method='trf',args=(Xexp, Yexp),verbose=2)

theta = Solution.x
print("theta = ",theta)
X_pred = X
Y_pred = model(theta, X_pred)

In [ ]:
# plt.plot(X,Y,'b-',linewidth=2,label=r"$y$")
plt.scatter(Xexp,Yexp, color = "r", s=20,label=r"$y_{exp}$")
plt.plot(X,Y,'r-',linewidth=1,label=r"$y_{true}$")
plt.plot(X_pred, Y_pred, linestyle = "--", color = "g", label=r"$NLR$")
# plt.plot(X,Y_gp,'b',linestyle = "--",label=r"$y_{gp}$", linewidth = 3)
# plt.fill_between(X, Y_gp-2*stdev_gp, Y_gp+2*stdev_gp, alpha=0.2, color = "b", label = "95% CI")
# plt.title(r"all trained")
plt.xlabel(r"$x$",fontsize=14)
plt.ylabel(r'$y$',fontsize=14)
plt.legend()
plt.show()

In [ ]:
# plt.plot(X,Y,'b-',linewidth=2,label=r"$y$")
plt.scatter(Xexp,Yexp, color = "r", s=20,label=r"$y$")
plt.plot(X,Y,'r-',linewidth=1,label=r"$f(\mathbf{\theta_{true}})$")
plt.plot(X_pred, Y_pred, linestyle = "--", color = "g", label= "NLR " + r"$f(\mathbf{\theta})$")
# plt.plot(X,Y_gp,'b',linestyle = "--",label=r"$y_{gp}$", linewidth = 3)
# plt.fill_between(X, Y_gp-2*stdev_gp, Y_gp+2*stdev_gp, alpha=0.2, color = "b", label = "95% CI")
# plt.title(r"all trained")
plt.xlabel(r"$x$",fontsize=14)
plt.ylabel(r'$y$',fontsize=14)
plt.legend()
plt.show()

In [ ]:
def plot_x_vs_y_given_theta2(data, exp_data, train_data, test_data, xbins, ybins, title, x_label, y_label, title_fontsize = 24, other_fontsize = 20, save_path = None):
    """
    Plots x data vs y data for any given parameter set theta
    
    Parameters
    ----------
    data: Instance of Data,
    
    """
    subplots_needed = data.get_dim_x_vals()
    fig, ax, num_subplots = create_subplots(subplots_needed)
    
    #Print the title and labels as appropriate
    set_plot_titles(fig, title, x_label, y_label, title_fontsize, other_fontsize)
        
    #Loop over different hyperparameters (number of subplots)
    for i in range(num_subplots):
        #If you still have data to plot
        if i < data.get_dim_x_vals():
            #The index of the data is i, and one data type is in the last row of the data
            X_space = data.x_vals[:,i]
            if not len(train_data.y_vals) == len(train_data.x_vals):            
                ax[i].plot(X_space, np.zeros(len(X_space)), label = r"$y_{true}$", color = "red")
                y_NLR = regression_func(theta, exp_data.x_vals[:,i], exp_data.y_vals)
                ax[i].plot(exp_data.x_vals[:,i], y_NLR, linestyle = "--", color = "g", label=r"$NLR$")    
            else:
                if exp_data is not None:
                    ax[i].scatter(exp_data.x_vals[:,i], exp_data.y_vals, color = "red", marker = "o", label = r"$y_{exp}$")
                ax[i].plot(X_space, data.y_vals, label = r"$y_{true}$", color = "red")            
                ax[i].plot(X_space, model(theta, X_space), linestyle = "--", color = "g", label=r"$NLR$")   
                
            ax[i].plot(X_space, data.gp_mean, lw=2, label=r"$y_{gp}$", color = "blue")
                
#             if train_data is not None:
#                 ax[i].scatter(train_data.x_vals[:,i], train_data.y_vals, color = "green",  s=150, marker = "x", label = "Training")
#             if test_data is not None:
#                 ax[i].scatter(test_data.x_vals[:,i], test_data.y_vals, color = "red", s=100, marker = "x", label = "Testing")
            
            ax[i].fill_between(
                X_space,
                data.gp_mean - 1.96 * np.sqrt(data.gp_var),
                data.gp_mean + 1.96 * np.sqrt(data.gp_var),
                alpha=0.3, label = "95% CI" )
            subplot_details(ax[i], X_space, None, None, None, None , xbins, ybins, other_fontsize)
        #Set axes off if it's an extra
        else:
            ax[i].set_axis_off()
            
        #Fetch handles and labels on last iteration
        if i == num_subplots-1:
            handles, labels = ax[i].get_legend_handles_labels()
            
    #Plots legend and title
    plt.tight_layout()
    fig.legend(handles, labels, loc= "upper left", fontsize = other_fontsize, bbox_to_anchor=(1.0, 0.95), borderaxespad=0)
    
    #save or show figure
    if save_path is None:
        plt.show()
        plt.close()
    else:
        save_fig(save_path, ext='png', close=True, verbose=False)  
    
    return

In [ ]:
#Plot xy data
path_list = ["2b0d1ee15911cd245f17a7d10a7abf3e",
             "b0e257d6e76025909e364903b3b792ee",
             "d0c8b17229671998f3ffdab06c79e1f9",
             "fdc2673ac83f3d0cf5b3b969a3e80825",
             "9042085b58a00b76d6c2ae0a20cfe9f3"]

df_best_path = "workspace/" + path_list[0] + "/ep_study_best_all.csv"
df_best = pd.read_csv(df_best_path, header = 0, index_col = 0)
run_num_list = list(map(int, df_best["Run Number"].to_numpy() + 1))
bo_iter_list = list(map(int, df_best["BO Iter"].to_numpy() + 1))

for i in range(len(path_list)):
    file_path = "workspace/" + path_list[i] + "/BO_Results.gz"
    
    run_num = run_num_list[i]
    bo_iter = bo_iter_list[i]
    x_lin_pts = 50

    title_fontsize = 24
    other_fontsize = 20
    save_path = None

    xbins = 5
    ybins = 5
    title = None
    x_label = r"X"
    y_label = r"$e(\theta)$"

    theta_opt_data, exp_data, train_data, test_data = analyze_xy_plot(file_path, run_num, bo_iter, x_lin_pts)
    plot_x_vs_y_given_theta2(theta_opt_data, exp_data, train_data, None, xbins, ybins, title, x_label, y_label, 
                            title_fontsize, other_fontsize, save_path)